<a href="https://colab.research.google.com/github/hwk06023/diff2lip/blob/main/inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Demo for "[Diff2Lip: Audio Conditioned Diffusion Models for Lip-Synchronization](https://soumik-kanad.github.io/diff2lip/)"
- Colab Notebook Author: Soumik Mukhopadhyay (soumik@umd.edu)

## Instructions:
- Change the runtime to GPU from top menu:  Runtime > Change runtime type > Hardware accelarator > T4 GPU.
- If you are not using google chrome, you may need to manually upload the videos (using the left side bar) and set the path manually in the "Set these paths manually if you upload your files manually" cell.
  - You can download the example video from [here](https://drive.google.com/file/d/1yhsa3WQZS22s4uHTwma2t0Q48ZYD5BzR/view?usp=sharing).


In [ ]:
!nvidia-smi

Thu Aug 29 03:11:55 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
#@title Clone repo
!git clone https://github.com/soumik-kanad/diff2lip.git

Cloning into 'diff2lip'...
remote: Enumerating objects: 93, done.
remote: Counting objects: 100% (93/93), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 93 (delta 17), reused 66 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (93/93), 15.74 MiB | 40.59 MiB/s, done.
Resolving deltas: 100% (17/17), done.


In [ ]:
#@title Install requirements
import os
os.chdir('diff2lip')
!pip install -r requirements.txt

Obtaining file:///content/diff2lip/guided-diffusion (from -r requirements.txt (line 11))
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.3/57.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.3/214.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 MB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.1/67.1 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 95.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 90.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.8/28.8 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 85.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 105.9 MB/s eta 0:00:00
   ━━━━━

In [ ]:
#@title Download Checkpoint
! mkdir -p checkpoints
! gdown 15vMcyPtFDEttNwelP3IouwsMpnccTqit -O checkpoints/checkpoint.pt

Downloading...
From (original): https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit
From (redirected): https://drive.google.com/uc?id=15vMcyPtFDEttNwelP3IouwsMpnccTqit&confirm=t&uuid=5e24821c-20e3-4a41-8f0f-0cb830390f96
To: /content/diff2lip/checkpoints/checkpoint.pt
100% 407M/407M [00:01<00:00, 208MB/s]


In [ ]:
#@title helper functions
from google.colab import files
import shutil
os.makedirs("data", exist_ok=True)

def upload_file():
    uploaded = files.upload()
    uploaded_paths = []
    for filename in uploaded.keys():
        dst_path = os.path.join("data", filename)
        print(f'move {filename} to {dst_path}')
        shutil.move(filename, dst_path)
        uploaded_paths.append(dst_path)
    return uploaded_paths

In [ ]:
#@title Upload Video
v = upload_file()
video_path=v[0]
audio_path=v[0]

Saving sample.mp4 to sample.mp4
move sample.mp4 to data/sample.mp4


In [ ]:
#@title Upload Audio (Upload again or skip this cell if the audio and video are present in the same video uploaded above)
a = upload_file()
audio_path=a[0]

Saving ye.mp3 to ye.mp3
move ye.mp3 to data/ye.mp3


In [ ]:
#@title Set these paths manually if you upload your files manually
#video_path="data/sample.mp4"
#audio_path="data/ye.mp3"
out_path="output.mp4"

In [ ]:
#@title Setting Flags
sample_mode="cross" # or "reconstruction"
generate_from_filelist=0
model_path="checkpoints/checkpoint.pt"
pads="0,0,0,0"

#cross vs reconstruction
if sample_mode == "reconstruction":
    sample_input_flags="--sampling_input_type=first_frame --sampling_ref_type=first_frame"
elif sample_mode == "cross":
    sample_input_flags="--sampling_input_type=gt --sampling_ref_type=gt"
else:
    print('Error: sample_mode can only be \"cross\" or \"reconstruction\"')

MODEL_FLAGS=f"--attention_resolutions 32,16,8 --class_cond False --learn_sigma True --num_channels 128 --num_head_channels 64 --num_res_blocks 2 --resblock_updown True --use_fp16 True --use_scale_shift_norm False"
DIFFUSION_FLAGS=f"--predict_xstart False  --diffusion_steps 1000 --noise_schedule linear --rescale_timesteps False"
SAMPLE_FLAGS=f"--sampling_seed=7   {sample_input_flags} --timestep_respacing ddim25 --use_ddim True --model_path={model_path}"
DATA_FLAGS=f"--nframes 5 --nrefer 1 --image_size 128 --sampling_batch_size=32 "
TFG_FLAGS=f"--face_hide_percentage 0.5 --use_ref=True --use_audio=True --audio_as_style=True"
GEN_FLAGS=f"--generate_from_filelist {generate_from_filelist}  --video_path={video_path} --audio_path={audio_path} --out_path={out_path} --save_orig=False --face_det_batch_size 16 --pads {pads} --is_voxceleb2=False"


In [ ]:
#@title Run Generation (took ~ 4 mins for the example video)
!python generate.py {MODEL_FLAGS}  {DIFFUSION_FLAGS}  {SAMPLE_FLAGS} {DATA_FLAGS} {TFG_FLAGS} {GEN_FLAGS}

2024-08-29 03:15:25.407207: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-29 03:15:25.427679: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-29 03:15:25.434020: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-29 03:15:25.449341: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-29 03:15:26.461760: W tensorflow/compiler/tf2

In [ ]:
#@title Display Output Video
import moviepy.editor
moviepy.editor.ipython_display("output.mp4")